## Getting a dated synthesis tree for an arbitrary set of taxa

## Standardizing query taxon names

One of the key challenges of comparing trees across studies is differences in taxon names because of spelling or taxonomic idiosincracies.

A solution to this, is mapping taxon names to unique identifiers using the Open Tree Taxonomic Name Resolution Service (TNRS). There are a few options to use this service including via the API, or the browser based bulk name mapping.

### Open Tree TNRS bulk name mapping tool.

Access this tool at https://tree.opentreeoflife.org/curator/tnrs/

This is a new beta-version of this functionality, so some parts are a bit finicky.

*Try this*
  * Click on "Add names..." (second button at the top of the menu on the left), and upload the names file `drosophila_example`. The "loading file" window will not close by itself, click the (X).
  * In the "Mapping options" section (bottom of the menu to the left):
    - select 'Insects' to narrow down the possibilities and speed up mapping
  * Click "Map selected names" (middle of the menu to the left).
  * Exact matches will show up in green, and can be accepted by clicking "accept exact matches".
  * Once you have accepted names for each of the taxa, click "Save nameset...", download it to your laptop, and extract (unzip) the files. You can take a look at the human readable version of the output at `output/main.csv`. `main.json` contains the the same data in a more computer readable format.
  * Finally, transfer the `main.csv` file to your working folder, so you can use it to get the tree for your taxa.

*Make sure your mappings were saved! If you do not **accept** matches (by clicking buttons), they do not download.*


In [121]:
import requests
import json
import sys
import dendropy
import csv
from opentree import OT


In [128]:
mapped_names = "../drosophila_example/drosophila_main.csv"


## uses the csv to create a dictionary with OTTids as keys, and the label you input as values
with open(mapped_names) as fp:
    reader = csv.reader(fp, delimiter=",", quotechar='"')
    next(reader, None)  # skip the headers
    label_dict = {'ott'+row[2]:row[0] for row in reader}

    
## Node ids contains the list of tip ids you want in your final tree
node_ids = [key for key in label_dict]


## Dated trees
To estimate dates, we will use the Chronosynth API. 
The dates API is work-in-progress, and so it is not yet as user friendly as it will be. 

A summary of the methods is here: https://github.com/OpenTreeOfLife/chronosynth/wiki/Chronosynth-methods-overview  
There are some API docs here: https://github.com/OpenTreeOfLife/chronosynth/wiki/Draft-API-docs

Using the dates API we can get dates that align to for individual nodes in the synth tree. 
This is based on the same information you saw in the conflict viewer.  
You can you a CURL call to GET the current information for dates for a node.

In [160]:
!curl -X GET https://dates.opentreeoflife.org/v4/dates/synth_node_age/ott109893 | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   860  100   860    0     0    794      0  0:00:01  0:00:01 --:--:--   795
{
  "query": "ott109893",
  "synth_node_id": "ott109893",
  "ot:source_node_ages": [
    {
      "age": 1.331418,
      "source_id": "ot_2072@tree1",
      "source_node": "node24"
    },
    {
      "age": 25.913621,
      "source_id": "ot_2018@tree8",
      "source_node": "node109"
    },
    {
      "age": 11.822624999999999,
      "source_id": "ot_809@tree2",
      "source_node": "node13685"
    },
    {
      "age": 28.474208,
      "source_id": "ot_409@tree2",
      "source_node": "node61812"
    },
    {
      "age": 28.303778,
      "source_id": "ot_409@tree1",
      "source_node": "node19094"
    },
    {
      "age": 13.074385,
      "source_id": "ot_2170@tree3",
      "source_node": "node234"
    },
    {
      "age": 13.18745,
      "source_i

To look at the node itself, you can navigate to 
"https://tree.opentreeoflife.org/curator/study/view/{STUDY ID}?tab=home&tree={TREE ID}&node={NODE_ID}"  
e.g. https://tree.opentreeoflife.org/curator/study/view/ot_2018?tab=home&tree=tree9&node=node1412  

In [164]:
## You can also get dates for arbitrary nodes in the synth tree, which are not associated with taxa.
!curl -X GET https://dates.opentreeoflife.org/v4/dates/synth_node_age/mrcaott1000311ott3643727 | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   184  100   184    0     0    211      0 --:--:-- --:--:-- --:--:--   211
{
  "query": "mrcaott1000311ott3643727",
  "synth_node_id": "mrcaott1000311ott3643727",
  "ot:source_node_ages": [
    {
      "age": 9.325001,
      "source_id": "ot_1592@tree1",
      "source_node": "node22956"
    }
  ]
}


This node https://tree.opentreeoflife.org/opentree/argus/opentree13.4@mrcaott1000311ott3643727 in the synthetic tree aligns with this node in this dated tree https://tree.opentreeoflife.org/curator/study/view/ot_1592?tab=home&tree=tree1&node=node22956

## Summarizing dates  for a set of taxa
You can use these dates individually, or use the API to get the dates across nodes in a tree, and then smooth the nodes in between

The approach in these examples uses rate smoothing, via bladj https://phylodiversity.net/phylocom/  
These approaches are rough, and there is NO infomation for many nodes, so be cautious!
### To run bladj we need a max age estimate for the root of the tree.

In [130]:
## Get the synth tree MRCA of the taxa of interest, so we know the root of our tree
output = OT.synth_mrca(node_ids=node_ids)
print(json.dumps(output.response_dict['mrca'], indent=4))



Unknown/unrecognized query ids (skipped):
 ott361374
ott4418979 


{
    "node_id": "ott34905",
    "num_tips": 4554,
    "supported_by": {
        "ott3.3draft1": "ott34905"
    },
    "taxon": {
        "name": "Drosophilidae",
        "ott_id": 34905,
        "rank": "family",
        "tax_sources": [
            "ncbi:7214",
            "worms:987176",
            "gbif:5547",
            "irmng:100842"
        ],
        "unique_name": "Drosophilidae"
    },
    "terminal": {
        "ot_1046@tree1": "node28",
        "ot_1047@tree1": "node63",
        "pg_1337@tree6167": "node1053390",
        "pg_2594@tree6014": "node1021794",
        "pg_2710@tree6291": "node1094072",
        "pg_2822@tree6569": "node1141999",
        "pg_437@tree6242": "node1081938"
    },
    "was_constrained": true,
    "was_uncontested": true
}


In [131]:
!curl -X GET https://dates.opentreeoflife.org/v4/dates/synth_node_age/ott34905 | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0     38      0  0:00:02  0:00:02 --:--:--    38
{
  "query": "ott34905",
  "synth_node_id": "ott34905",
  "ot:source_node_ages": null
}


In [165]:
## You can also get node dates for a node in python, ucing the 'requests' module
datesurl     = 'https://dates.opentreeoflife.org/v4/dates/synth_node_age/{}'.format(output.response_dict['mrca']['node_id'])
resp = requests.get(datesurl)
resp.json()

{'query': 'ott34905', 'synth_node_id': 'ott34905', 'ot:source_node_ages': None}

We do not have an age for this node in the database, so we need to use external information.  
We will use 50 Million years for the age of drosophilidae, based on https://www.sciencedirect.com/topics/agricultural-and-biological-sciences/drosophilidae

In [168]:
## Get Dated synth tree
# datesurl     = 'https://dates.opentreeoflife.org/v4/dates/dated_tree'
url =  'http://127.0.0.1:1983/v4/dates/dated_tree'

payload = { "node_ids" : node_ids,
            "max_age": max_age}

max_age = 50
resp = requests.post(url=url, data=json.dumps(payload))
resp_dict = resp.json()



This returns a response object, 

In [170]:
resp_dict.keys()

dict_keys(['dated_trees_newick_list', 'topology_sources', 'date_sources', 'tar_file_download'])

In [177]:
dated_tree = dendropy.Tree.get(data=resp_dict['dated_trees_newick_list'][0], schema="newick")
dated_tree.write(path="ottid_dated_tree.tre", schema='newick')


In [171]:
resp_dict['tar_file_download']

'dates.opentreeoflife.org/v4/dates/download_dates_tar/chrono_out_01_12_2023_07_28_37.tar.gz'

In [174]:
## To get the full set of ages, 
! wget http://127.0.0.1:1983/v4/dates/download_dates_tar/chrono_out_01_12_2023_07_28_37.tar.gz


--2023-01-12 07:30:23--  http://127.0.0.1:1983/v4/dates/download_dates_tar/chrono_out_01_12_2023_07_28_37.tar.gz
Connecting to 127.0.0.1:1983... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5078 (5.0K) [application/gzip]
Saving to: ‘chrono_out_01_12_2023_07_28_37.tar.gz’

chrono_out_01_12_20 100%[===================>]   4.96K  --.-KB/s    in 0s      

2023-01-12 07:30:23 (409 MB/s) - ‘chrono_out_01_12_2023_07_28_37.tar.gz’ saved [5078/5078]



### Evaluate your confidence in your tree.
How many nodes are in your tree? How many do you have date estimates for?

## We can also compare nodes in our custom synth tree to dated trees, and use them to infer dates for those nodes.

In [116]:
## Read in your custom synth tree

import dendropy
custom_synth_dir = "snacktavish_dros_34905_tmp5exp6dql"
treepath = "{}/labelled_supertree/labelled_supertree.tre".format(custom_synth_dir)
custom_synth = dendropy.Tree.get_from_path(treepath, schema = "newick")


In [117]:
#Sometimes there are subspecies, when your query is actually species.
collapsed_tax = []
for node in custom_synth:
    if not node.is_leaf():
        if node.label in label_dict:
            print(node.label)
            print(label_dict[node.label])
            collapsed_tax.append(node.label)
            node.clear_child_nodes()
            node.taxon = custom_synth.taxon_namespace.new_taxon(label=node.label)


leaves_A = [tip.taxon.label for tip in custom_synth.leaf_node_iter()]
print("Total number of tips in synth tree after collapsing queries with lower level taxa is {}\n".format(len(leaves_A)))


ott1082279
Drosophila mojavensis
ott138603
Drosophila mercatorum
ott534107
Drosophila americana
ott287233
Drosophila sulfurigaster
ott245830
Drosophila nasuta
ott1082272
Drosophila pseudoobscura
ott616768
Drosophila paulistorum
ott616777
Drosophila tropicalis
ott505710
Scaptodrosophila lebanonensis
Total number of tips in synth tree after collapsing queries with lower level taxa is 5396



In [180]:
taxa_to_retain = list(label_dict.keys())
custom_synth.retain_taxa_with_labels(taxa_to_retain)

In [119]:
leaves_B = [tip.taxon.label for tip in custom_synth.leaf_node_iter()]

In [178]:

## Get Dated synth tree
#datesurl     = 'https://dates.opentreeoflife.org/v4/dates/dated_tree'
url =  'http://127.0.0.1:1983/v4/dates/dated_tree'

payload = { "newick" : custom_synth.as_string(schema="newick"),
            "max_age": max_age}
max_age = 50
resp = requests.post(url=url, data=json.dumps(payload))
resp_dict = json.loads(resp.content.decode())

if resp.status_code != 200:
    sys.stderr.write(resp.content.decode())
    sys.exit()


dated_tree = dendropy.Tree.get(data=resp_dict['dated_trees_newick_list'][0], schema="newick")
#dated_tree.write(path=output_dir+"/ottid_dated_tree.tre", schema='newick')

In [179]:
resp_dict.keys()

dict_keys(['dated_trees_newick_list', 'topology_sources', 'date_sources', 'tar_file_download'])

### Assessing uncertainty

Lets choose a part of the synth tree where we have lot of data!  
https://tree.opentreeoflife.org/opentree/opentree13.4@ott109893/Cracidae  
We can estimate a dated tree for Cracidae

In [187]:
## Get Dated synth tree
url     = 'https://dates.opentreeoflife.org/v4/dates/dated_tree'
#url =  'http://127.0.0.1:1983/v4/dates/dated_tree'

payload = { "node_id" : 'ott109893',
            "reps" : 5}

resp = requests.post(url=url, data=json.dumps(payload))
resp_dict = resp.json()


In [188]:
resp_dict

{'dated_trees_newick_list': ['((((((((ott75785:0.224579,ott157739:0.224579)mrcaott75785ott157739:0.224579,(ott157735:0.224579,ott188770:0.224579)mrcaott157735ott188770:0.224579)mrcaott75785ott157735:0.350996,ott157745:0.800154)mrcaott75785ott157745:0.782351,(ott963344:0.791253)ott109896:0.791253)mrcaott75785ott963344:0.445519,ott1014142:2.028024)mrcaott75785ott1014142:3.635540,(((ott109884:1.509077,ott658355:1.509077)mrcaott109884ott658355:1.919686,ott157741:3.428763)mrcaott109884ott157741:0.806121,((ott109886:2.159300,ott188772:2.159300)mrcaott109886ott188772:1.350991,(((ott157743:0.566530,ott1024688:0.566530)mrcaott157743ott1024688:0.855687,ott1014136:1.422217)mrcaott157743ott1014136:1.363479,ott712301:2.785697)mrcaott157743ott712301:0.724594)mrcaott109886ott157743:0.724594,ott174442:4.234884,ott1024690:4.234884,ott3596020:4.234884,ott3596021:4.234884,ott3596022:4.234884,ott3596023:4.234884,ott6151696:4.234884,ott6151699:4.234884,ott7659705:4.234884,ott7659706:4.234884,ott7659707:4.2